In [4]:
import pandas as pd


In [1]:

# Create sample election data
data = {
    'Constituency': ['District1', 'District1', 'District1',
                    'District2', 'District2', 'District2',
                    'District3', 'District3', 'District3'],
    'Party': ['Party A', 'Party B', 'Party C',
             'Party A', 'Party B', 'Party C',
             'Party A', 'Party B', 'Party C'],
    'Candidate': ['John Smith', 'Jane Doe', 'Bob Wilson',
                 'Alice Brown', 'Mike Jones', 'Sarah Davis',
                 'Tom White', 'Lucy Black', 'Peter Green'],
    'Votes': [1500, 1200, 800,
             2000, 2200, 1800,
             1700, 1600, 1400]
}



In [2]:
# Create DataFrame
df = pd.DataFrame(data)

def calculate_total_votes_per_party():
    """Calculate total votes received by each party across all constituencies"""
    return df.groupby('Party')['Votes'].sum()

def get_winning_party(constituency_data):
    """Find the party with maximum votes in a constituency"""
    max_votes_idx = constituency_data['Votes'].idxmax()
    return constituency_data.loc[max_votes_idx, 'Party']

def find_constituency_winners():
    """Identify winning party in each constituency"""
    return df.groupby('Constituency').apply(get_winning_party)

def determine_overall_winner():
    """Find the party with the highest total votes"""
    total_votes = calculate_total_votes_per_party()
    return total_votes.idxmax()

def calculate_vote_share():
    """Calculate vote share percentages for each candidate"""
    total_votes_per_constituency = df.groupby('Constituency')['Votes'].sum()
    df['Vote_Share'] = df.apply(
        lambda x: (x['Votes'] / total_votes_per_constituency[x['Constituency']]) * 100,
        axis=1
    )
    return df

def identify_close_contests(margin_threshold=12):
    """
    Identify constituencies with close contests
    (vote margin between top two candidates less than threshold%)
    """
    close_contests = []

    for constituency in df['Constituency'].unique():
        constituency_data = df[df['Constituency'] == constituency].copy()
        constituency_data = constituency_data.sort_values('Votes', ascending=False)

        if len(constituency_data) >= 2:
            top_votes = constituency_data.iloc[0]['Votes']
            second_votes = constituency_data.iloc[1]['Votes']
            total_votes = constituency_data['Votes'].sum()

            margin = ((top_votes - second_votes) / total_votes) * 100

            if margin < margin_threshold:
                close_contests.append({
                    'Constituency': constituency,
                    'Margin': margin,
                    'Winner': constituency_data.iloc[0]['Party'],
                    'Runner_up': constituency_data.iloc[1]['Party']
                })

    return pd.DataFrame(close_contests)


In [3]:

# Main analysis
if __name__ == "__main__":
    print("\nTotal votes per party:")
    print(calculate_total_votes_per_party())

    print("\nConstituency winners:")
    print(find_constituency_winners())

    print("\nOverall winner:")
    print(determine_overall_winner())

    print("\nVote shares:")
    print(calculate_vote_share()[['Constituency', 'Party', 'Candidate', 'Votes', 'Vote_Share']])

    print("\nClose contests (margin < 12%):")
    print(identify_close_contests())


Total votes per party:
Party
Party A    5200
Party B    5000
Party C    4000
Name: Votes, dtype: int64

Constituency winners:
Constituency
District1    Party A
District2    Party B
District3    Party A
dtype: object

Overall winner:
Party A

Vote shares:


<ipython-input-2-faadb364e52e>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('Constituency').apply(get_winning_party)


  Constituency    Party    Candidate  Votes  Vote_Share
0    District1  Party A   John Smith   1500   42.857143
1    District1  Party B     Jane Doe   1200   34.285714
2    District1  Party C   Bob Wilson    800   22.857143
3    District2  Party A  Alice Brown   2000   33.333333
4    District2  Party B   Mike Jones   2200   36.666667
5    District2  Party C  Sarah Davis   1800   30.000000
6    District3  Party A    Tom White   1700   36.170213
7    District3  Party B   Lucy Black   1600   34.042553
8    District3  Party C  Peter Green   1400   29.787234

Close contests (margin < 12%):
  Constituency    Margin   Winner Runner_up
0    District1  8.571429  Party A   Party B
1    District2  3.333333  Party B   Party A
2    District3  2.127660  Party A   Party B
